## 全连接神经网络（多层感知机）

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
# import data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#test data
testX = mnist.test.images
testY = mnist.test.labels

In [4]:
#初始化权重
def weight_variable(shape):
    data = tf.truncated_normal(stddev=0.01,shape=shape)
    return tf.Variable(data)

#初始化偏置单元
def bais_variable(shape):
    data = tf.constant(0.01,shape=shape)
    return tf.Variable(data)

In [5]:
#一层神经元
class Layer:
    def __init__(self,n_input,n_output,transfer_function=tf.nn.softmax,pre_layer=None):
        #self.W = tf.Variable(tf.zeros([n_input,n_output]))
        #self.b = tf.Variable(tf.zeros([n_output]))
        self.W = weight_variable([n_input,n_output])
        self.b = bais_variable([n_output])
        self.transfer_function = transfer_function
        self.pre_layer = pre_layer
    
    def transform(self,X):
        if self.pre_layer is not None:
            X = self.pre_layer.transform(X)
        return self.transfer_function(tf.matmul(X ,self.W)+self.b)
    
input_layer = Layer(784,200)
output_layer = Layer(200,10,pre_layer=input_layer)

X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,10])

model = output_layer.transform(X)

lossF = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model),reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(Y,1),tf.argmax(model,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

train_step = tf.train.AdamOptimizer(0.1).minimize(lossF)


In [6]:
def show_accuracy(testX,testY):
    print("accuracy=",sess.run(accuracy,{X:testX,Y:testY}))

In [7]:
#记录loss,W,b
tf.summary.scalar('loss_0.1_784_200_10_500_1000',lossF)
tf.summary.scalar('accuracy_0.1_784_200_10_500_1000',accuracy)
merged = tf.summary.merge_all()

In [8]:
sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

train_data = mnist.train

trainWriter = tf.summary.FileWriter( 'logs/MNIST_NN/train',sess.graph)
testWriter = tf.summary.FileWriter( 'logs/MNIST_NN/test',sess.graph)

for i in range(500):
    batch_x , batch_y = mnist.train.next_batch(1000)
    c , _ ,summaried = sess.run([lossF,train_step,merged] , feed_dict={X:batch_x,Y:batch_y})
    if i%10==0:
        trainWriter.add_summary(summaried,i)
        testSummary = sess.run(merged,feed_dict={X:testX,Y:testY})
        testWriter.add_summary(testSummary,i)
        print("steps %d cost:%.3f" % (i,c) )

show_accuracy(testX,testY)

steps 0 cost:2.303
steps 10 cost:1.398
steps 20 cost:0.696
steps 30 cost:0.444
steps 40 cost:0.484
steps 50 cost:0.422
steps 60 cost:0.424
steps 70 cost:0.369
steps 80 cost:0.353
steps 90 cost:0.313
steps 100 cost:0.332
steps 110 cost:0.312
steps 120 cost:0.330
steps 130 cost:0.319
steps 140 cost:0.368
steps 150 cost:0.382
steps 160 cost:0.334
steps 170 cost:0.390
steps 180 cost:0.346
steps 190 cost:0.404
steps 200 cost:0.310
steps 210 cost:0.268
steps 220 cost:0.391
steps 230 cost:0.323


KeyboardInterrupt: 